In [1]:
import requests
import os
import dotenv
import pandas as pd

Finding Python API information here: https://open-meteo.com/en/docs?latitude=19.7297&longitude=-155.09&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,showers,evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,wind_gusts_10m

Directions: 
1) install openmeteo-requests
2) install requests-cache retry-requests

imports needed to access the API:
```python
import pandas as pd
import requests_cache
from retry_requests import retry
```

Example code template given from the API_docs page:
```python
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 19.7297,
	"longitude": -155.09,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "showers", "evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_80m", "wind_speed_120m", "wind_speed_180m", "wind_direction_10m", "wind_direction_80m", "wind_direction_120m", "wind_direction_180m", "wind_gusts_10m"],
}
responses = openmeteo.weather_api(url, params=params)
```

In [3]:
# following the tutorial above, starting with appropriate imports:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

In [8]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 19.7297,
	"longitude": -155.09,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "showers", "evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_80m", "wind_speed_120m", "wind_speed_180m", "wind_direction_10m", "wind_direction_80m", "wind_direction_120m", "wind_direction_180m", "wind_gusts_10m"],
}
responses = openmeteo.weather_api(url, params=params)

Everything seems to work! Not familiar with this API yet, doing some exploring:

In [9]:
# what format is responses in?
type(responses)

list

In [14]:
# check the length of the list and look at the first item
print(len(responses))
responses[0]

1


Ah, ok! So it seems the template builds it this way so that one could put in different cities and get a list of different responses. In this case, there is only one. So we can just convert that single response to json for our data.

In [29]:
# set data to our response for Hilo, Hawaii
data = responses[0]

# now I can access any of the parameters we called for - in this example hourly data - the first variable is relative humidity 2m
hourly = data.Hourly()
hourly.Variables(1).ValuesAsNumpy()

array([ 66.,  70.,  73.,  76.,  82.,  89.,  90.,  78.,  65.,  68.,  70.,
        69.,  69.,  70.,  70.,  69.,  69.,  71.,  71.,  61.,  64.,  62.,
        61.,  61.,  66.,  70.,  74.,  78.,  85.,  93.,  94.,  96.,  88.,
        79.,  76.,  75.,  75.,  74.,  73.,  73.,  73.,  73.,  66.,  65.,
        65.,  60.,  55.,  55.,  58.,  67.,  71.,  72.,  79.,  86.,  88.,
        87.,  83.,  80.,  75.,  71.,  73.,  77.,  79.,  80.,  82.,  83.,
        78.,  77.,  72.,  68.,  66.,  68.,  71.,  76.,  81.,  86.,  91.,
        95.,  95.,  95.,  95.,  94.,  94.,  94.,  94.,  95.,  97.,  96.,
        89.,  80.,  72.,  68.,  66.,  65.,  65.,  65.,  67.,  71.,  77.,
        82.,  87.,  91.,  95.,  98., 100., 100.,  95.,  88.,  82.,  80.,
        80.,  80.,  82.,  85.,  84.,  77.,  66.,  58.,  54.,  52.,  53.,
        58.,  65.,  73.,  81.,  88.,  92.,  88.,  81.,  75.,  75.,  77.,
        78.,  77.,  76.,  75.,  77.,  80.,  81.,  77.,  71.,  66.,  64.,
        63.,  64.,  68.,  73.,  79.,  85.,  91.,  9

All in all this API is very clear and easy to use, with extremely helpful documentation. The code below was generated through the OpenMeteo API docs website after giving it the parameters I wanted to check for. 

In [30]:
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_showers = hourly.Variables(4).ValuesAsNumpy()
hourly_evapotranspiration = hourly.Variables(5).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_speed_80m = hourly.Variables(8).ValuesAsNumpy()
hourly_wind_speed_120m = hourly.Variables(9).ValuesAsNumpy()
hourly_wind_speed_180m = hourly.Variables(10).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(11).ValuesAsNumpy()
hourly_wind_direction_80m = hourly.Variables(12).ValuesAsNumpy()
hourly_wind_direction_120m = hourly.Variables(13).ValuesAsNumpy()
hourly_wind_direction_180m = hourly.Variables(14).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(15).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["evapotranspiration"] = hourly_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_80m"] = hourly_wind_speed_80m
hourly_data["wind_speed_120m"] = hourly_wind_speed_120m
hourly_data["wind_speed_180m"] = hourly_wind_speed_180m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_80m"] = hourly_wind_direction_80m
hourly_data["wind_direction_120m"] = hourly_wind_direction_120m
hourly_data["wind_direction_180m"] = hourly_wind_direction_180m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.head()

,date,temperature_2m,relative_humidity_2m,precipitation,rain,showers,evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,wind_gusts_10m
0,2025-09-24 00:00:00+00:00,23.056499,66.0,0.0,0.0,0.0,0.28,0.958428,14.529915,19.346441,20.111729,20.124611,48.012871,44.999897,44.274860,42.825249,31.319998
1,2025-09-24 01:00:00+00:00,22.406500,70.0,0.0,0.0,0.0,0.20,0.813021,14.264361,19.855560,20.371038,20.898613,47.045418,44.999897,43.567978,42.207226,34.560001
2,2025-09-24 02:00:00+00:00,21.856501,73.0,0.0,0.0,0.0,0.15,0.707639,12.768586,18.504139,19.083395,19.469976,40.425995,37.092915,35.789059,33.690102,33.119999
3,2025-09-24 03:00:00+00:00,21.256500,76.0,0.0,0.0,0.0,0.11,0.606373,11.808878,17.673029,18.775303,19.201874,37.568665,33.366394,32.471172,30.411011,29.879999
4,2025-09-24 04:00:00+00:00,20.106501,82.0,0.0,0.0,0.0,0.04,0.423724,9.339208,16.267857,17.253731,18.118410,27.552727,24.863682,23.355589,20.955858,26.639999
